# Fix&Furn Mini - Kaggle Replay
This notebook rebuilds the Fix&Furn Mini chatbot from scratch, pulling the IKEA Saudi dataset directly from Kaggle before launching the Gradio demo.


## Workflow Overview
- Install dependencies and configure API keys
- Connect to Kaggle and download the IKEA Saudi dataset
- Scaffold the local assets required by the app
- Write the project modules with inline walkthroughs
- Launch the Gradio demo for live testing


In [ ]:
# Install Kaggle API and app dependencies
!pip install --quiet kaggle google-generativeai gradio python-dotenv reportlab


## 1. Configure keys
Set up Gemini and Kaggle credentials for this notebook session.


In [ ]:
import os
import getpass

if 'GEMINI_API_KEY' not in os.environ or not os.environ['GEMINI_API_KEY'].strip():
    os.environ['GEMINI_API_KEY'] = getpass.getpass('Enter your Gemini API key: ')
print('Gemini key configured.')


### Kaggle API credentials
Create the ~/.kaggle/kaggle.json file required by the Kaggle CLI.


In [ ]:
import os
import json
import getpass
from pathlib import Path

kaggle_dir = Path.home() / '.kaggle'
kaggle_dir.mkdir(exist_ok=True)

kaggle_username = input('Enter your Kaggle username: ').strip()
kaggle_key = getpass.getpass('Enter your Kaggle API key: ').strip()

kaggle_creds = {'username': kaggle_username, 'key': kaggle_key}
with open(kaggle_dir / 'kaggle.json', 'w') as f:
    json.dump(kaggle_creds, f)

try:
    os.chmod(kaggle_dir / 'kaggle.json', 0o600)
except PermissionError:
    pass

print('Kaggle credentials configured.')


## 2. Download the IKEA Saudi dataset
Fetch the furniture catalog snapshot that powers the IKEA lookup tool.


In [ ]:
from pathlib import Path

data_dir = Path('fixnfurn_mini/data')
data_dir.mkdir(parents=True, exist_ok=True)

!kaggle datasets download -d ahmedkallam/ikea-sa-furniture-web-scraping -p fixnfurn_mini/data --unzip

zip_path = data_dir / 'ikea-sa-furniture-web-scraping.zip'
if zip_path.exists():
    zip_path.unlink()
print('Dataset ready.')


## 3. Scaffold Fix&Furn Mini assets
Write the curated catalog, repair pricing rules, and business prompts needed by the app.


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path('fixnfurn_mini')
DATA_DIR = PROJECT_ROOT / 'data'
ME_DIR = PROJECT_ROOT / 'me'
LOGS_DIR = PROJECT_ROOT / 'logs'

for folder in (PROJECT_ROOT, DATA_DIR, ME_DIR, LOGS_DIR):
    folder.mkdir(exist_ok=True)

catalog_json = "[\n  {\n    \"name\": \"Atlas Dining Table\",\n    \"sku\": \"AT-120-OAK\",\n    \"category\": \"Table\",\n    \"dimensions_cm\": {\n      \"W\": 180,\n      \"D\": 90,\n      \"H\": 75\n    },\n    \"material_primary\": \"Solid oak top, steel legs\",\n    \"color_options\": [\n      \"Natural Oak\",\n      \"Walnut Stain\",\n      \"Black Base\"\n    ],\n    \"price_usd\": 649,\n    \"stock_qty\": 7,\n    \"warranty_months\": 12,\n    \"assembly_required\": false,\n    \"notes\": \"Oil finish; seats 6\\u20138; available with matching bench.\"\n  },\n  {\n    \"name\": \"Nora 3-Seater Sofa\",\n    \"sku\": \"NR-300-FAB\",\n    \"category\": \"Sofa\",\n    \"dimensions_cm\": {\n      \"W\": 210,\n      \"D\": 90,\n      \"H\": 84\n    },\n    \"material_primary\": \"Kiln-dried wood frame, foam cushions\",\n    \"color_options\": [\n      \"Linen Sand\",\n      \"Charcoal\",\n      \"Forest Green\"\n    ],\n    \"price_usd\": 899,\n    \"stock_qty\": 3,\n    \"warranty_months\": 12,\n    \"assembly_required\": true,\n    \"notes\": \"Removable cushion covers; add-on ottoman available.\"\n  },\n  {\n    \"name\": \"Milo Glass Coffee Table\",\n    \"sku\": \"ML-080-GLS\",\n    \"category\": \"Coffee Table\",\n    \"dimensions_cm\": {\n      \"W\": 120,\n      \"D\": 60,\n      \"H\": 40\n    },\n    \"material_primary\": \"Tempered glass top, ash wood base\",\n    \"color_options\": [\n      \"Clear Glass / Natural Ash\",\n      \"Smoked Glass / Black Ash\"\n    ],\n    \"price_usd\": 279,\n    \"stock_qty\": 12,\n    \"warranty_months\": 12,\n    \"assembly_required\": false,\n    \"notes\": \"Replacement glass available by order.\"\n  },\n  {\n    \"name\": \"Ava Bookshelf\",\n    \"sku\": \"AV-150-ASH\",\n    \"category\": \"Bookshelf\",\n    \"dimensions_cm\": {\n      \"W\": 80,\n      \"D\": 35,\n      \"H\": 180\n    },\n    \"material_primary\": \"Laminated ash veneer over MDF, steel frame\",\n    \"color_options\": [\n      \"Natural Ash\",\n      \"Walnut\",\n      \"Matte Black\"\n    ],\n    \"price_usd\": 349,\n    \"stock_qty\": 10,\n    \"warranty_months\": 12,\n    \"assembly_required\": true,\n    \"notes\": \"Anti-tip kit included; adjustable shelves.\"\n  },\n  {\n    \"name\": \"Luna Writing Desk\",\n    \"sku\": \"LN-110-WAL\",\n    \"category\": \"Desk\",\n    \"dimensions_cm\": {\n      \"W\": 120,\n      \"D\": 60,\n      \"H\": 75\n    },\n    \"material_primary\": \"Walnut veneer top, steel frame\",\n    \"color_options\": [\n      \"Walnut / Black\",\n      \"Walnut / White\"\n    ],\n    \"price_usd\": 399,\n    \"stock_qty\": 6,\n    \"warranty_months\": 12,\n    \"assembly_required\": true,\n    \"notes\": \"Cable grommet included; optional drawer module.\"\n  }\n]"
(DATA_DIR / 'catalog.json').write_text(catalog_json + '\n', encoding='utf-8')

price_rules_json = "{\n  \"scratch\": {\n    \"wood\": {\n      \"small\": [\n        55,\n        95,\n        2,\n        3\n      ],\n      \"medium\": [\n        95,\n        160,\n        3,\n        4\n      ],\n      \"large\": [\n        160,\n        240,\n        4,\n        6\n      ]\n    },\n    \"glass\": {\n      \"small\": [\n        65,\n        110,\n        3,\n        4\n      ],\n      \"medium\": [\n        110,\n        180,\n        4,\n        6\n      ],\n      \"large\": [\n        180,\n        260,\n        5,\n        7\n      ]\n    },\n    \"metal\": {\n      \"small\": [\n        60,\n        120,\n        2,\n        3\n      ],\n      \"medium\": [\n        120,\n        190,\n        3,\n        5\n      ],\n      \"large\": [\n        190,\n        280,\n        4,\n        6\n      ]\n    }\n  },\n  \"broken_glass\": {\n    \"any\": {\n      \"small\": [\n        210,\n        320,\n        4,\n        6\n      ],\n      \"medium\": [\n        320,\n        480,\n        6,\n        9\n      ],\n      \"large\": [\n        480,\n        720,\n        7,\n        12\n      ]\n    }\n  },\n  \"wobble\": {\n    \"any\": {\n      \"small\": [\n        95,\n        160,\n        2,\n        3\n      ],\n      \"medium\": [\n        160,\n        260,\n        3,\n        5\n      ],\n      \"large\": [\n        260,\n        420,\n        4,\n        7\n      ]\n    }\n  },\n  \"loose_joint\": {\n    \"any\": {\n      \"small\": [\n        85,\n        140,\n        2,\n        3\n      ],\n      \"medium\": [\n        140,\n        220,\n        3,\n        4\n      ],\n      \"large\": [\n        220,\n        360,\n        4,\n        6\n      ]\n    }\n  },\n  \"hinge_alignment\": {\n    \"any\": {\n      \"small\": [\n        60,\n        110,\n        1,\n        2\n      ],\n      \"medium\": [\n        90,\n        140,\n        1,\n        2\n      ],\n      \"large\": [\n        130,\n        200,\n        2,\n        3\n      ]\n    }\n  },\n  \"drawer_stick\": {\n    \"any\": {\n      \"small\": [\n        60,\n        110,\n        1,\n        2\n      ],\n      \"medium\": [\n        90,\n        140,\n        1,\n        2\n      ],\n      \"large\": [\n        130,\n        200,\n        2,\n        3\n      ]\n    }\n  },\n  \"upholstery_tear\": {\n    \"fabric\": {\n      \"small\": [\n        140,\n        260,\n        5,\n        7\n      ],\n      \"medium\": [\n        260,\n        420,\n        6,\n        9\n      ],\n      \"large\": [\n        420,\n        680,\n        7,\n        12\n      ]\n    }\n  },\n  \"refinish\": {\n    \"wood\": {\n      \"small\": [\n        320,\n        520,\n        7,\n        10\n      ],\n      \"medium\": [\n        520,\n        820,\n        9,\n        12\n      ],\n      \"large\": [\n        820,\n        1200,\n        12,\n        15\n      ]\n    }\n  },\n  \"repaint\": {\n    \"metal\": {\n      \"small\": [\n        140,\n        220,\n        4,\n        6\n      ],\n      \"medium\": [\n        220,\n        360,\n        5,\n        7\n      ],\n      \"large\": [\n        360,\n        520,\n        6,\n        9\n      ]\n    }\n  }\n}"
(DATA_DIR / 'price_rules.json').write_text(price_rules_json + '\n', encoding='utf-8')

system_prompt_text = "You are Fix&Furn Mini - a friendly furniture sales and repair concierge for a one-day demo.\nUse ONLY details from business_summary.txt, about_business.pdf, the curated catalog, repair rules, and tool outputs.\n\nGoals:\n- For product questions, call lookup_product with the customer's keyword, SKU, color, category, or IKEA item ID. Summarize the best matches, mentioning item_id + name (and SKU when available), key specs, price, colors, and stock info. Present the IKEA-sourced items as part of our Fix&Furn partner lineup.\n- For repair quotes, call estimate_repair with issue, material (if known), and size_category. Present budget vs standard vs rush tiers with price and turnaround ranges.\n- When a customer wants to buy or book, confirm name + email + a short note, then call record_customer_interest exactly once.\n- If the question is outside your resources, call record_feedback so the team can follow up manually.\n- After resolving a purchase or repair conversation, offer to log feedback (service quality, satisfaction) via record_service_feedback.\n\nDefinitions:\n- size_category in {small, medium, large}: small < 40 cm side; medium 40-120 cm; large > 120 cm.\n- Common issues: scratch, broken_glass, wobble, loose_joint, hinge_alignment, drawer_stick, upholstery_tear, refinish, repaint.\n\nStyle:\n- Be concise, transparent, and practical.\n- Clarify whether an item is from our in-house Fix&Furn selection or our IKEA Partner Line (treat the IKEA data as trusted partner inventory).\n- If data is missing, say so and offer next steps (e.g., inspection or sourcing confirmation).\n- Always quote prices in USD. For IKEA items, note the conversion (1 SAR \u2248 0.2667 USD) when relevant.\n- Always include any known warranty or lead-time notes that apply.\n- When collecting feedback, keep it short (sentiment + comments) and thank the customer.\n"
(ME_DIR / 'system_prompt.txt').write_text(system_prompt_text, encoding='utf-8')

business_summary_text = "Fix&Furn Mini is a micro furniture shop and repair concierge you can demo in one day.\n\nScope:\n- Answer product questions about the in-house catalog (dimensions, materials, color options, price, stock, warranty).\n- Reference an IKEA Saudi Arabia dataset (positioned as our Fix&Furn x IKEA Partner Line) to suggest comparable items by item_id, name, price, dimensions, colors, and online availability.\n- IKEA list prices are converted from SAR at roughly 1 SAR ~= 0.2667 USD so conversations stay in USD.\n- Estimate repair price and turnaround ranges for common issues (scratches, broken glass, loose joints, hinge or drawer alignment, minor upholstery tears, refinishing, repainting). Pricing bands blend the 2023 workshop rate card with 2025 national averages.\n- Capture qualified leads (name, email, intent) and log unanswered questions for follow-up.\n- Collect post-service satisfaction feedback (product purchases, repairs, delivery experience).\n\nOperating assumptions:\n- The Fix&Furn showroom highlights five hero products with known SKUs, inventory counts, and 12-month warranty.\n- IKEA data is used as an external benchmark only; final pricing and availability must be confirmed with suppliers.\n- Standard workshop turnaround is 2-5 days; glass or fabric special orders run 4-10 days.\n- Repairs carry a 30-day workmanship warranty.\n- Service coverage stays within city limits, with pickup/delivery available for a quoted fee outside the chatbot.\n"
(ME_DIR / 'business_summary.txt').write_text(business_summary_text, encoding='utf-8')

print('Scaffold complete.')


## 4. Write core modules
Generate the two Python modules that power the concierge.


### tools.py
Utility layer for catalog search, repair pricing, and lightweight JSONL logging.


In [ ]:
%%writefile fixnfurn_mini/tools.py
from datetime import datetime
from pathlib import Path
import csv
import json
import re
from typing import Dict, List

BASE = Path(__file__).parent
LOGS = BASE / "logs"
DATA = BASE / "data"
LOGS.mkdir(exist_ok=True)

CATALOG = json.loads((DATA / "catalog.json").read_text(encoding="utf-8"))
PRICE_RULES = json.loads((DATA / "price_rules.json").read_text(encoding="utf-8"))
IKEA_CSV = DATA / "IKEA_SA_Furniture_Web_Scrapings_sss.csv"
SAR_TO_USD = 0.2667  # rough mid-market exchange rate


def _to_float(value: str):
    if value is None:
        return None
    value = value.strip()
    if not value or value.lower().startswith("no "):
        return None
    try:
        return float(value)
    except ValueError:
        return None


def _to_bool(value: str):
    if value is None:
        return None
    value = value.strip().lower()
    if value in {"true", "yes", "y", "1"}:
        return True
    if value in {"false", "no", "n", "0"}:
        return False
    return None


def _load_ikea_catalog() -> List[Dict]:
    if not IKEA_CSV.exists():
        fallback = BASE / "IKEA_SA_Furniture_Web_Scrapings_sss.csv"
        if not fallback.exists():
            return []
        target = fallback
    else:
        target = IKEA_CSV

    items: List[Dict] = []
    with target.open("r", encoding="utf-8", newline="") as fh:
        reader = csv.DictReader(fh)
        for row in reader:
            item_id = (row.get("item_id") or "").strip()
            name = (row.get("name") or "").strip()
            if not item_id or not name:
                continue
            category = (row.get("category") or "").strip()
            price_sar = _to_float(row.get("price"))
            price_usd = round(price_sar * SAR_TO_USD, 2) if price_sar is not None else None
            width = _to_float(row.get("width"))
            height = _to_float(row.get("height"))
            depth = _to_float(row.get("depth"))
            short_description = (row.get("short_description") or "").strip()
            if short_description:
                short_description = re.sub(r"\s+", " ", short_description)
            other_colors = (row.get("other_colors") or "").strip()
            if other_colors.lower() in {"no", "n/a"}:
                other_colors = ""
            sellable = _to_bool(row.get("sellable_online"))
            link = (row.get("link") or "").strip()
            designer = (row.get("designer") or "").strip()

            searchable = " ".join(
                filter(
                    None,
                    [
                        item_id.lower(),
                        name.lower(),
                        category.lower(),
                        short_description.lower(),
                        other_colors.lower(),
                        designer.lower(),
                    ],
                )
            )

            items.append(
                {
                    "item_id": item_id,
                    "name": name,
                    "category": category,
                    "price_usd": price_usd,
                    "price_currency": "USD" if price_usd is not None else None,
                    "price_note": (
                        f"Converted from SAR at 1 SAR = {SAR_TO_USD:.4f} USD"
                        if price_usd is not None
                        else None
                    ),
                    "sellable_online": sellable,
                    "link": link,
                    "other_colors": other_colors,
                    "short_description": short_description,
                    "designer": designer,
                    "dimensions_cm": {
                        k: v
                        for k, v in {"width": width, "height": height, "depth": depth}.items()
                        if v is not None
                    },
                    "_search": searchable,
                }
            )
    return items


IKEA_ITEMS = _load_ikea_catalog()


def _copy_public_item(item: Dict) -> Dict:
    return {k: v for k, v in item.items() if not k.startswith("_")}


def _search_ikea_items(query: str, limit: int = 5) -> List[Dict]:
    if not IKEA_ITEMS:
        return []
    q = query.strip().lower()
    if not q:
        return []
    words = [w for w in re.split(r"\W+", q) if w]
    scored: Dict[str, List] = {}
    for item in IKEA_ITEMS:
        score = 0
        if q == item["item_id"].lower():
            score += 10
        if q in item["_search"]:
            score += 3
        if words:
            score += sum(1 for w in words if w and w in item["_search"])
        if score > 0:
            existing = scored.get(item["item_id"])
            if existing is None or score > existing[0]:
                scored[item["item_id"]] = [score, item]
    if not scored:
        return []
    top = sorted(scored.values(), key=lambda pair: (-pair[0], pair[1]["name"]))
    return [_copy_public_item(item) for _, item in top[:limit]]


def record_customer_interest(email: str, name: str, message: str):
    entry = {"ts": datetime.utcnow().isoformat(), "email": email, "name": name, "message": message}
    out = LOGS / "leads.jsonl"
    with out.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(entry) + "\n")
    print(f"[LEAD] {entry}")
    return {"ok": True, "msg": "Thanks! We'll follow up soon."}


def record_feedback(question: str):
    entry = {"ts": datetime.utcnow().isoformat(), "question": question}
    out = LOGS / "feedback.jsonl"
    with out.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(entry) + "\n")
    print(f"[FEEDBACK] {entry}")
    return {"ok": True, "msg": "Noted. We'll improve our answers."}


def record_service_feedback(
    email: str,
    name: str,
    service_type: str,
    satisfaction: str,
    comments: str = "",
):
    entry = {
        "ts": datetime.utcnow().isoformat(),
        "email": email,
        "name": name,
        "service_type": service_type,
        "satisfaction": satisfaction,
        "comments": comments or "",
    }
    out = LOGS / "service_feedback.jsonl"
    with out.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(entry) + "\n")
    print(f"[SERVICE_FEEDBACK] {entry}")
    return {"ok": True, "msg": "Thanks for the feedback! We'll share it with the team."}


def lookup_product(query: str):
    q = (query or "").strip()
    if not q:
        return {"ok": False, "msg": "Please provide a product keyword, SKU, or IKEA item ID."}
    q_lower = q.lower()
    result = {"ok": True, "query": q}

    sku_match = next((item for item in CATALOG if item.get("sku", "").lower() == q_lower), None)
    if sku_match:
        result["catalog_match"] = sku_match

    name_hits = [
        item
        for item in CATALOG
        if q_lower in item.get("name", "").lower()
        or any(q_lower in (opt or "").lower() for opt in item.get("color_options", []))
    ]
    if name_hits and not sku_match:
        result["catalog_results"] = name_hits

    category_hits = [item for item in CATALOG if item.get("category", "").lower() == q_lower]
    if category_hits:
        result["catalog_category"] = category_hits

    ikea_hits = _search_ikea_items(q_lower)
    if ikea_hits:
        result["ikea_results"] = ikea_hits

    if len(result) == 2:
        return {"ok": False, "msg": f"No products found for '{q}'."}
    return result


def estimate_repair(issue: str, material: str = "any", size_category: str = "medium"):
    issue = issue.strip().lower()
    material = (material or "any").strip().lower()
    size = (size_category or "medium").strip().lower()
    rules = PRICE_RULES.get(issue)
    if not rules:
        return {"ok": False, "msg": f"No pricing rule for issue '{issue}'."}
    if material in rules:
        bucket = rules[material]
    elif "any" in rules:
        bucket = rules["any"]
    else:
        bucket = next(iter(rules.values()))
    if size not in bucket:
        return {"ok": False, "msg": f"Unsupported size_category '{size}'. Use small/medium/large."}
    min_p, max_p, min_d, max_d = bucket[size]
    tiers = {
        "budget": {"price": round(min_p * 0.9), "days": [min_d, max(min_d, min_d + 1)]},
        "standard": {"price": round((min_p + max_p) / 2), "days": [min_d, max_d]},
        "rush": {"price": round(max_p * 1.25), "days": [max(1, min_d - 1), max(1, max_d - 1)]},
    }
    return {"ok": True, "issue": issue, "material": material, "size": size, "estimate": tiers}


### app.py
Gradio chat interface plus Gemini function-calling orchestration.


In [ ]:
%%writefile fixnfurn_mini/app.py
import os
from pathlib import Path

import gradio as gr
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai import types
from google.protobuf.json_format import MessageToDict
from google.protobuf.struct_pb2 import ListValue as ProtoListValue, Struct as ProtoStruct, Value as ProtoValue

from tools import (
    estimate_repair,
    lookup_product,
    record_customer_interest,
    record_feedback,
    record_service_feedback,
)

load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise RuntimeError("GEMINI_API_KEY is not set. Create a .env file based on .env.example.")

genai.configure(api_key=API_KEY)

MODEL_NAME = os.getenv("GEMINI_MODEL", "gemini-2.5-flash")
SYSTEM_PROMPT = Path("me/system_prompt.txt").read_text(encoding="utf-8")
SUMMARY = Path("me/business_summary.txt").read_text(encoding="utf-8")

FUNCTION_DECLARATIONS = [
    {
        "name": "lookup_product",
        "description": (
            "Search the Fix&Furn curated catalog and IKEA Saudi Arabia reference dataset. "
            "Return relevant catalog_match/catalog_results and ikea_results including item_id, name, "
            "category, price_sar, dimensions_cm, availability, and link when available."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Keyword, color, category, SKU, or IKEA item ID to search for.",
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "estimate_repair",
        "description": "Estimate repair price and turnaround tiers based on issue, material, and size_category.",
        "parameters": {
            "type": "object",
            "properties": {
                "issue": {
                    "type": "string",
                    "description": "Issue such as scratch, broken_glass, wobble, loose_joint, hinge_alignment, drawer_stick, upholstery_tear, refinish, repaint.",
                },
                "material": {
                    "type": "string",
                    "description": "Primary material (wood, glass, metal, fabric, or any).",
                },
                "size_category": {
                    "type": "string",
                    "description": "Furniture size bucket: small, medium, or large.",
                },
            },
            "required": ["issue"],
        },
    },
    {
        "name": "record_customer_interest",
        "description": "Capture customer details when they are ready to buy or book a repair.",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "Customer email address."},
                "name": {"type": "string", "description": "Customer full name."},
                "message": {"type": "string", "description": "Short note about the product or repair request."},
            },
            "required": ["email", "name", "message"],
        },
    },
    {
        "name": "record_feedback",
        "description": "Log customer questions that the assistant could not resolve.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {"type": "string", "description": "Unanswered or unclear customer request."}
            },
            "required": ["question"],
        },
    },
    {
        "name": "record_service_feedback",
        "description": "Capture post-service feedback about the overall experience, product satisfaction, or repair quality.",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "Customer email to match the service record."},
                "name": {"type": "string", "description": "Customer full name."},
                "service_type": {
                    "type": "string",
                    "description": "What we delivered (e.g., purchase, repair, delivery, install).",
                },
                "satisfaction": {
                    "type": "string",
                    "description": "Quick sentiment summary (e.g., happy, neutral, unhappy, 1-5).",
                },
                "comments": {
                    "type": "string",
                    "description": "Optional free-text feedback on the experience.",
                },
            },
            "required": ["email", "name", "service_type", "satisfaction"],
        },
    },
]

MODEL = genai.GenerativeModel(
    model_name=MODEL_NAME,
    system_instruction=f"{SYSTEM_PROMPT}\n\n{SUMMARY}",
    tools=[{"function_declarations": FUNCTION_DECLARATIONS}],
)

GENERATION_CONFIG = types.GenerationConfig(temperature=0.2)
TOOL_CONFIG = {"function_calling_config": {"mode": "AUTO"}}


def _content(role: str, text: str):
    if not text:
        return None
    return {"role": role, "parts": [{"text": text}]}


def _convert_history(history):
    converted = []
    for user, assistant in history:
        if user:
            converted.append(_content("user", user))
        if assistant:
            converted.append(_content("model", assistant))
    return [msg for msg in converted if msg is not None]


def _call_tool(name: str, args: dict):
    try:
        if name == "lookup_product":
            return lookup_product(**args)
        if name == "estimate_repair":
            return estimate_repair(**args)
        if name == "record_customer_interest":
            return record_customer_interest(**args)
        if name == "record_feedback":
            return record_feedback(**args)
        if name == "record_service_feedback":
            return record_service_feedback(**args)
        return {"ok": False, "msg": f"Unknown tool '{name}'."}
    except TypeError as exc:
        return {"ok": False, "msg": f"Invalid arguments for {name}: {exc}"}


def _first_function_call(response):
    for candidate in response.candidates or []:
        if not candidate or not candidate.content:
            continue
        for part in candidate.content.parts:
            if part.function_call:
                return part.function_call
    return None


def _proto_to_python(value):
    if value is None:
        return None
    if isinstance(value, ProtoValue):
        kind = value.WhichOneof("kind")
        if kind == "struct_value":
            return {k: _proto_to_python(v) for k, v in value.struct_value.fields.items()}
        if kind == "list_value":
            return [_proto_to_python(v) for v in value.list_value.values]
        if kind == "string_value":
            return value.string_value
        if kind == "number_value":
            return value.number_value
        if kind == "bool_value":
            return value.bool_value
        if kind == "null_value":
            return None
        return MessageToDict(value, preserving_proto_field_name=True)
    if isinstance(value, ProtoStruct):
        return {k: _proto_to_python(v) for k, v in value.fields.items()}
    if isinstance(value, ProtoListValue):
        return [_proto_to_python(v) for v in value.values]
    if isinstance(value, dict):
        return {k: _proto_to_python(v) for k, v in value.items()}
    if hasattr(value, "items"):
        return {k: _proto_to_python(v) for k, v in value.items()}
    if isinstance(value, (list, tuple)):
        return [_proto_to_python(v) for v in value]
    return value


def _function_args_to_dict(function_call):
    args = getattr(function_call, "args", None)
    if args is None:
        return {}
    if isinstance(args, dict):
        return {k: _proto_to_python(v) for k, v in args.items()}
    if hasattr(args, "_pb"):
        try:
            return _proto_to_python(args._pb)
        except Exception:
            pass
    if hasattr(args, "ListFields"):
        try:
            return _proto_to_python(args)
        except Exception:
            pass
    if hasattr(args, "items"):
        return {k: _proto_to_python(v) for k, v in args.items()}
    converted = _proto_to_python(args)
    return converted if isinstance(converted, dict) else {}


def _send_function_response(chat, name: str, payload: dict):
    message = {
        "role": "tool",
        "parts": [
            {
                "function_response": {
                    "name": name,
                    "response": payload,
                }
            }
        ],
    }
    return chat.send_message(message, generation_config=GENERATION_CONFIG, tool_config=TOOL_CONFIG)


def chat_fn(message, history):
    history_msgs = _convert_history(history)
    chat = MODEL.start_chat(history=history_msgs)
    response = chat.send_message(
        message,
        generation_config=GENERATION_CONFIG,
        tool_config=TOOL_CONFIG,
    )

    while True:
        function_call = _first_function_call(response)
        if not function_call:
            break

        args = _function_args_to_dict(function_call)
        tool_result = _call_tool(function_call.name, args)
        response = _send_function_response(chat, function_call.name, tool_result)

    text = response.text or ""
    return text.strip()


TITLE = "Fix&Furn Mini - Furniture Sales & Repair Concierge"
demo = gr.ChatInterface(chat_fn, title=TITLE)

if __name__ == "__main__":
    demo.launch()


## 5. Launch the Gradio demo
Import the app module and start the interface. Use `share=True` when you need a public link.


In [ ]:
import sys
from pathlib import Path

project_dir = Path('fixnfurn_mini').resolve()
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

from app import demo

demo.launch(share=True)
